## Imports

In [11]:
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime, timedelta

## Config

In [23]:
RAW_BASE = "../datasets/circuit-of-the-americas/COTA/Race 2"
OUTPUT_ROOT = os.path.abspath("../datasets_clean/cota2")
os.makedirs(OUTPUT_ROOT, exist_ok=True)

ANALYSIS_FILE = os.path.join(RAW_BASE, "23_AnalysisEnduranceWithSections_ Race 2_Anonymized.csv")
TELEMETRY_FILE = os.path.join(RAW_BASE, "R2_cota_telemetry_data.csv")

## Utility

In [24]:
def time_to_seconds(t):
    """Convert strings like 'M:SS.mmm' or 'H:MM:SS.mmm' or 'MM:SS' to seconds (float).
    Returns NaN for missing or unparsable inputs."""
    if pd.isna(t):
        return np.nan
    s = str(t).strip()
    if s == '':
        return np.nan
    # replace comma decimal separators
    s = s.replace(',', '.')
    parts = s.split(':')
    try:
        if len(parts) == 1:
            return float(parts[0])
        if len(parts) == 2:
            m, sec = parts
            return float(m) * 60.0 + float(sec)
        if len(parts) == 3:
            h, m, sec = parts
            return float(h) * 3600.0 + float(m) * 60.0 + float(sec)
    except Exception:
        return np.nan

def clean_column_names(df):
    df = df.copy()
    df.columns = [c.strip() for c in df.columns]
    return df

## Cleaning time

In [25]:
# ---------------------- 1) Load & clean "analysis with sections" ----------------------
print("Loading analysis file:", ANALYSIS_FILE)
analysis_df = pd.read_csv(ANALYSIS_FILE, sep=';', dtype=str)
analysis_df = clean_column_names(analysis_df)
print("Raw columns:", analysis_df.columns.tolist())

# Drop fully-empty columns
analysis_df = analysis_df.dropna(how='all', axis=1)

dup_counts = (
    analysis_df.groupby(["NUMBER", "LAP_NUMBER"])
    .size()
    .reset_index(name="count")
)

# Find driver NUMBERs with duplicate laps
ambiguous_numbers = dup_counts.loc[dup_counts["count"] > 1, "NUMBER"].unique().tolist()

print(f"🚨 Found ambiguous driver NUMBERs (duplicate laps): {ambiguous_numbers}")

# Remove those drivers from analysis_df
before_analysis = len(analysis_df)
analysis_df = analysis_df[~analysis_df["NUMBER"].isin(ambiguous_numbers)].copy()
after_analysis = len(analysis_df)

print(f"🧹 Removed {before_analysis - after_analysis} rows from analysis.csv")

# Drop unwanted columns to make files readable in Excel
for drop_col in ['CLASS', 'GROUP', 'MANUFACTURER']:
    if drop_col in analysis_df.columns:
        analysis_df = analysis_df.drop(columns=[drop_col])

# Convert numeric-ish columns
for col in ['NUMBER', 'DRIVER_NUMBER', 'LAP_NUMBER', 'KPH', 'TOP_SPEED', 'PIT_TIME']:
    if col in analysis_df.columns:
        analysis_df[col] = pd.to_numeric(analysis_df[col], errors='coerce')

# Convert LAP times and sector strings to seconds
for col in ['LAP_TIME', 'S1', 'S2', 'S3', 'S1_SECONDS', 'S2_SECONDS', 'S3_SECONDS']:
    if col in analysis_df.columns:
        # prefer S?_SECONDS if present numeric; otherwise convert textual S1 etc.
        if col.endswith('_SECONDS'):
            analysis_df[col] = pd.to_numeric(analysis_df[col], errors='coerce')
        else:
            analysis_df[col + '_SEC'] = analysis_df[col].apply(time_to_seconds)

# If explicit seconds columns exist but not the _SEC ones, copy them
for s_col in ['S1_SECONDS', 'S2_SECONDS', 'S3_SECONDS']:
    if s_col in analysis_df.columns and (s_col.replace('_SECONDS', '') + '_SEC') not in analysis_df.columns:
        analysis_df[s_col.replace('_SECONDS', '') + '_SEC'] = pd.to_numeric(analysis_df[s_col], errors='coerce')

# Normalize flag column
if 'FLAG_AT_FL' in analysis_df.columns:
    analysis_df['FLAG_AT_FL'] = analysis_df['FLAG_AT_FL'].astype(str).str.strip()

# Create a numeric Lap_Time if not present
if 'LAP_TIME_SEC' not in analysis_df.columns and 'LAP_TIME' in analysis_df.columns:
    analysis_df['LAP_TIME_SEC'] = analysis_df['LAP_TIME'].apply(time_to_seconds)

print("Loaded analysis rows:", len(analysis_df))

# ---------------------- 2) Filter out unwanted laps ----------------------
# Remove laps under FCY/SC/AUTO flags and laps that finished in pit
bad_flags = set(['FCY', 'SC', 'YELLOW'])
if 'FLAG_AT_FL' in analysis_df.columns:
    before = len(analysis_df)
    analysis_df = analysis_df[~analysis_df['FLAG_AT_FL'].isin(bad_flags)]
    print(f"Filtered by flag: {before} -> {len(analysis_df)}")

# Proper pit filtering: 'B' marks a pit entry lap, '0' means normal lap
if 'CROSSING_FINISH_LINE_IN_PIT' in analysis_df.columns:
    before = len(analysis_df)
    analysis_df['CROSSING_FINISH_LINE_IN_PIT'] = analysis_df['CROSSING_FINISH_LINE_IN_PIT'].astype(str).str.strip()
    analysis_df = analysis_df[analysis_df['CROSSING_FINISH_LINE_IN_PIT'] != 'B']
    print(f"Filtered pit-crossing laps: {before} -> {len(analysis_df)}")

# Remove clearly invalid lap times
if 'LAP_TIME_SEC' in analysis_df.columns:
    before = len(analysis_df)
    analysis_df = analysis_df[pd.to_numeric(analysis_df['LAP_TIME_SEC'], errors='coerce') > 5.0] # remove zeros and extremely small
    analysis_df = analysis_df[pd.to_numeric(analysis_df['LAP_TIME_SEC'], errors='coerce') < 3600.0] # sanity upper bound
    print(f"Filtered by lap time sanity: {before} -> {len(analysis_df)}")

# ---------------------- 3) Save cleaned per-driver CSVs (compact columns) ----------------------
# Select a useful subset of columns for driver-coaching CSVs
default_cols = [
'NUMBER','DRIVER_NUMBER','LAP_NUMBER','LAP_TIME','LAP_TIME_SEC',
'S1','S1_SEC','S2','S2_SEC','S3','S3_SEC',
'KPH','TOP_SPEED','ELAPSED','HOUR','FLAG_AT_FL'
]
cols_to_save = [c for c in default_cols if (c in analysis_df.columns and 'IMPROVEMENT' not in c.upper())]
print('Columns saved per driver:', cols_to_save)

# Ensure NUMBER is numeric for filename safety
if 'NUMBER' in analysis_df.columns:
    analysis_df['NUMBER'] = pd.to_numeric(analysis_df['NUMBER'], errors='coerce')

for num, grp in analysis_df.groupby('NUMBER'):
    safe_num = int(num) if not pd.isna(num) else 'unknown'
    out_path = os.path.join(OUTPUT_ROOT, f"driver_{safe_num}.csv")
    grp[cols_to_save].to_csv(out_path, index=False, sep=',')
    print(f"Saved driver file: {out_path} ({len(grp)} laps)")

# ---------------------- 4) Load telemetry (if present) and pivot into wide form ----------------------
if os.path.exists(TELEMETRY_FILE):
    print('Loading telemetry (this may be large):', TELEMETRY_FILE)
    tel_df = pd.read_csv(TELEMETRY_FILE)
    tel_df = clean_column_names(tel_df)

    # Normalize vehicle_number (handles '03' → 3)
    tel_df["vehicle_number"] = (
        tel_df["vehicle_number"]
        .astype(str)
        .str.extract(r"(\d+)$")[0]  # pull out digits at end
        .astype(float)
        .astype("Int64")
    )

    # Normalize ambiguous_numbers too
    ambiguous_numbers = [int(n) for n in ambiguous_numbers if pd.notna(n)]

    # Then filter
    before_telemetry = len(tel_df)
    tel_df = tel_df[~tel_df["vehicle_number"].isin(ambiguous_numbers)].copy()
    after_telemetry = len(tel_df)

    print(f"🧹 Removed {before_telemetry - after_telemetry} rows from telemetry.csv")

    # Parse times
    for tcol in ['meta_time','timestamp']:
        if tcol in tel_df.columns:
            tel_df[tcol] = pd.to_datetime(tel_df[tcol], errors='coerce', utc=True)
    # Ensure values numeric when possible
    tel_df['telemetry_value'] = pd.to_numeric(tel_df['telemetry_value'], errors='coerce')

    # Pivot: make each telemetry_name a column per vehicle_id + timestamp
    index_cols = [c for c in ['vehicle_id','meta_time','timestamp','lap'] if c in tel_df.columns]
    print('Telemetry index cols:', index_cols)
    tel_wide = tel_df.pivot_table(index=index_cols, columns='telemetry_name', values='telemetry_value', aggfunc='first').reset_index()
    print('Telemetry wide shape:', tel_wide.shape)

    # Save per-vehicle telemetry sampled per-lap timestamp rows (if lap info exists)
    if 'vehicle_id' in tel_wide.columns:
        telem_out = os.path.join(OUTPUT_ROOT, 'telemetry_per_timestamp.csv')
        tel_wide.to_csv(telem_out, index=False)
        print('Saved telemetry per-timestamp wide file to:', telem_out)
else:
    print('No telemetry file found at', TELEMETRY_FILE)

# ---------------------- 5) Compute per-lap telemetry summaries (vectorized) ----------------------
print("Generating per-lap and per-sector telemetry summary for MAD-filtered laps...")

if os.path.exists(TELEMETRY_FILE):
    tel_df = pd.read_csv(TELEMETRY_FILE)
    tel_df = clean_column_names(tel_df)
    tel_df['timestamp'] = pd.to_datetime(tel_df['timestamp'], errors='coerce', utc=True)
    tel_df['vehicle_id'] = tel_df['vehicle_id'].astype(str)
    tel_df = tel_df.dropna(subset=['timestamp', 'telemetry_value'])
    tel_df['telemetry_value'] = pd.to_numeric(tel_df['telemetry_value'], errors='coerce')

    # Keep only valid laps from driver_session_stats
    driver_stats_path = os.path.join(OUTPUT_ROOT, "driver_session_stats.csv")
    if os.path.exists(driver_stats_path):
        valid_laps_df = pd.read_csv(driver_stats_path)
        valid_pairs = analysis_df.merge(
            valid_laps_df[['DriverNumber']],
            left_on='NUMBER', right_on='DriverNumber',
            how='inner'
        )
        valid_laps = set(zip(valid_pairs['NUMBER'].astype(str), valid_pairs['LAP_NUMBER']))
        tel_df = tel_df[tel_df.apply(lambda r: (r['vehicle_id'], r.get('lap', np.nan)) in valid_laps, axis=1)]

    # Compute per-lap, per-sector metrics
    per_lap_list = []
    for (vid, lap), group in tel_df.groupby(['vehicle_id','lap']):
        lap_metrics = {'vehicle_id': vid, 'lap': lap}

        # Whole-lap metrics
        lap_metrics['mean_throttle'] = group.loc[group['telemetry_name'].isin(['ath','aps','throttle']),'telemetry_value'].mean()
        lap_metrics['mean_brake'] = group.loc[group['telemetry_name'].isin(['pbrake_f','pbrake_r','brake']),'telemetry_value'].mean()
        sa = group.loc[group['telemetry_name']=='Steering_Angle','telemetry_value']
        lap_metrics['steering_smoothness'] = sa.diff().abs().mean() if not sa.empty else np.nan

        # Optional: sector metrics if sector telemetry columns exist
        for sector in ['S1','S2','S3']:
            sector_mask = group['telemetry_name'].str.contains(sector, case=False)
            if sector_mask.any():
                lap_metrics[f'{sector}_mean_throttle'] = group.loc[group['telemetry_name'].isin(['ath','aps','throttle']) & sector_mask,'telemetry_value'].mean()
                lap_metrics[f'{sector}_mean_brake'] = group.loc[group['telemetry_name'].isin(['pbrake_f','pbrake_r','brake']) & sector_mask,'telemetry_value'].mean()
                sa_sector = group.loc[sector_mask & (group['telemetry_name']=='Steering_Angle'),'telemetry_value']
                lap_metrics[f'{sector}_steering_smoothness'] = sa_sector.diff().abs().mean() if not sa_sector.empty else np.nan

        per_lap_list.append(lap_metrics)

    if per_lap_list:
        per_lap_df = pd.DataFrame(per_lap_list)
        per_lap_path = os.path.join(OUTPUT_ROOT, 'per_lap_telemetry_summary.csv')
        per_lap_df.to_csv(per_lap_path, index=False)
        print('✅ Saved per-lap telemetry summary to:', per_lap_path)
else:
    print('⚠️ No telemetry data matched any valid MAD-filtered laps.')

# ---------------------- 6) Quick driver session statistics (filter extreme laps) ----------------------
print("\nComputing driver session statistics with sanity filtering...")

# Ensure numeric
for col in ['LAP_TIME_SEC', 'S1_SEC', 'S2_SEC', 'S3_SEC']:
    if col in analysis_df.columns:
        analysis_df[col] = pd.to_numeric(analysis_df[col], errors='coerce')

# Filter obviously invalid or extreme laps (more than 15 sec slower than driver mean)
if 'NUMBER' in analysis_df.columns:
    filtered_df_list = []
    for driver, grp in analysis_df.groupby('NUMBER'):
        grp = grp.copy()
        median = grp['LAP_TIME_SEC'].median()
        mad = (grp['LAP_TIME_SEC'] - median).abs().median()
        cutoff = median + 4 * mad  # 5 MADs above median

        # Debug info
        print(f"\nDriver {driver}:")
        print(f"  Number of laps before filtering: {len(grp)}")
        print(f"  Median lap time: {median:.3f} sec")
        print(f"  MAD (Median Abs Deviation): {mad:.3f} sec")
        print(f"  Cutoff for valid laps: {cutoff:.3f} sec")

        # Apply filtering
        filtered_grp = grp[grp['LAP_TIME_SEC'] <= cutoff]
        print(f"  Number of laps after filtering: {len(filtered_grp)}")
        if len(filtered_grp) < len(grp):
            removed = len(grp) - len(filtered_grp)
            print(f"  → Removed {removed} lap(s) as outliers.")

        filtered_df_list.append(filtered_grp)

    valid_laps_df = pd.concat(filtered_df_list, ignore_index=True)
else:
    valid_laps_df = analysis_df.copy()

# Compute per-driver stats
agg_dict = {
    'LAP_TIME_SEC': ['count','min','mean','std'],
    'S1_SEC': 'min', 'S2_SEC': 'min', 'S3_SEC': 'min'
}
driver_stats_df = valid_laps_df.groupby('NUMBER', as_index=False).agg(agg_dict)

# Flatten columns
driver_stats_df.columns = [
    'DriverNumber','Laps','BestLap(s)','AvgLap(s)','StdDev(s)',
    'S1Best','S2Best','S3Best'
]
driver_stats_df['TheoreticalBest(s)'] = driver_stats_df[['S1Best','S2Best','S3Best']].sum(axis=1)

# Round
driver_stats_df[['BestLap(s)','AvgLap(s)','StdDev(s)','TheoreticalBest(s)']] = \
    driver_stats_df[['BestLap(s)','AvgLap(s)','StdDev(s)','TheoreticalBest(s)']].round(3)

# Save CSV
driver_stats_path = os.path.join(OUTPUT_ROOT, "driver_session_stats.csv")
driver_stats_df.to_csv(driver_stats_path, index=False)
print(f"✅ Saved driver session stats to: {driver_stats_path}")
display(driver_stats_df)

Loading analysis file: ../datasets/circuit-of-the-americas/COTA/Race 2\23_AnalysisEnduranceWithSections_ Race 2_Anonymized.csv
Raw columns: ['NUMBER', 'DRIVER_NUMBER', 'LAP_NUMBER', 'LAP_TIME', 'LAP_IMPROVEMENT', 'CROSSING_FINISH_LINE_IN_PIT', 'S1', 'S1_IMPROVEMENT', 'S2', 'S2_IMPROVEMENT', 'S3', 'S3_IMPROVEMENT', 'KPH', 'ELAPSED', 'HOUR', 'S1_LARGE', 'S2_LARGE', 'S3_LARGE', 'TOP_SPEED', 'PIT_TIME', 'CLASS', 'GROUP', 'MANUFACTURER', 'FLAG_AT_FL', 'S1_SECONDS', 'S2_SECONDS', 'S3_SECONDS', 'IM1a_time', 'IM1a_elapsed', 'IM1_time', 'IM1_elapsed', 'IM2a_time', 'IM2a_elapsed', 'IM2_time', 'IM2_elapsed', 'IM3a_time', 'IM3a_elapsed', 'FL_time', 'FL_elapsed']
🚨 Found ambiguous driver NUMBERs (duplicate laps): ['3']
🧹 Removed 33 rows from analysis.csv
Loaded analysis rows: 475
Filtered by flag: 475 -> 359
Filtered by lap time sanity: 359 -> 359
Columns saved per driver: ['NUMBER', 'DRIVER_NUMBER', 'LAP_NUMBER', 'LAP_TIME', 'LAP_TIME_SEC', 'S1', 'S1_SEC', 'S2', 'S2_SEC', 'S3', 'S3_SEC', 'KPH', 'T

,DriverNumber,Laps,BestLap(s),AvgLap(s),StdDev(s),S1Best,S2Best,S3Best,TheoreticalBest(s)
0,2,7,148.609,149.418,0.729,32.364,56.755,59.300,148.419
1,5,10,149.848,151.368,1.301,32.396,57.624,59.060,149.080
2,7,10,148.112,148.388,0.260,32.424,56.699,58.876,147.999
3,8,10,153.233,155.357,1.668,32.959,58.737,60.815,152.511
4,11,10,151.501,153.305,1.226,33.118,57.951,59.704,150.773
5,13,10,148.115,148.708,0.535,32.408,56.630,58.819,147.857
6,14,9,149.906,151.669,1.155,32.654,57.623,59.466,149.743
7,15,10,149.400,151.474,1.492,32.340,57.214,59.660,149.214
8,16,10,148.510,149.428,0.665,32.225,56.791,59.316,148.332
9,18,6,151.905,152.783,0.447,33.042,58.123,60.395,151.560


In [26]:
# ---------------------- Filter extreme laps using MAD per driver ----------------------
if 'NUMBER' in analysis_df.columns:
    filtered_df_list = []
    driver_filter_summary = []  # keep record for debugging/inspection

    for driver, grp in analysis_df.groupby('NUMBER'):
        grp = grp.copy()
        median = grp['LAP_TIME_SEC'].median()
        mad = (grp['LAP_TIME_SEC'] - median).abs().median()
        cutoff = median + 3 * mad  # 3 MADs above median

        # Filter outliers
        filtered_grp = grp[grp['LAP_TIME_SEC'] <= cutoff]

        # Collect summary info
        driver_filter_summary.append({
            'DriverNumber': driver,
            'TotalLaps': len(grp),
            'ValidLaps': len(filtered_grp),
            'RemovedLaps': len(grp) - len(filtered_grp),
            'MedianLap(s)': median,
            'MAD(s)': mad,
            'Cutoff(s)': cutoff
        })

        print(f"\nDriver {driver}:")
        print(f"  Total laps: {len(grp)} | Valid: {len(filtered_grp)} | Removed: {len(grp) - len(filtered_grp)}")
        print(f"  Median: {median:.3f}s | MAD: {mad:.3f}s | Cutoff: {cutoff:.3f}s")

        # Save filtered laps to individual CSV
        safe_num = int(driver) if not pd.isna(driver) else 'unknown'
        out_path = os.path.join(OUTPUT_ROOT, f"driver_{safe_num}.csv")
        filtered_grp.to_csv(out_path, index=False)
        print(f"  ✅ Saved filtered driver file: {out_path} ({len(filtered_grp)} valid laps)")

        filtered_df_list.append(filtered_grp)

    # Combine all valid laps for session-level stats
    valid_laps_df = pd.concat(filtered_df_list, ignore_index=True)

    # Optional summary overview
    filter_summary_df = pd.DataFrame(driver_filter_summary)
    print("\nSummary of filtering across drivers:")
    display(filter_summary_df)
else:
    valid_laps_df = analysis_df.copy()

# ---------------------- Compute per-driver statistics (only valid laps) ----------------------
agg_dict = {
    'LAP_TIME_SEC': ['count','min','mean','std'],
    'S1_SEC': 'min', 'S2_SEC': 'min', 'S3_SEC': 'min'
}

driver_stats_df = valid_laps_df.groupby('NUMBER', as_index=False).agg(agg_dict)
driver_stats_df.columns = [
    'DriverNumber','Laps','BestLap(s)','AvgLap(s)','StdDev(s)',
    'S1Best','S2Best','S3Best'
]
driver_stats_df['TheoreticalBest(s)'] = driver_stats_df[['S1Best','S2Best','S3Best']].sum(axis=1)

# Round for readability
driver_stats_df[['BestLap(s)','AvgLap(s)','StdDev(s)','TheoreticalBest(s)']] = \
    driver_stats_df[['BestLap(s)','AvgLap(s)','StdDev(s)','TheoreticalBest(s)']].round(3)

# Save updated session-level stats
driver_stats_path = os.path.join(OUTPUT_ROOT, "driver_session_stats.csv")
driver_stats_df.to_csv(driver_stats_path, index=False)
print(f"\n✅ Saved filtered driver session stats to: {driver_stats_path}")
display(driver_stats_df)



Driver 2:
  Total laps: 9 | Valid: 7 | Removed: 2
  Median: 149.715s | MAD: 0.922s | Cutoff: 152.481s
  ✅ Saved filtered driver file: c:\Users\jacks\OneDrive\Desktop\progamming projects\toyota gr analysis\datasets_clean\cota2\driver_2.csv (7 valid laps)

Driver 5:
  Total laps: 13 | Valid: 10 | Removed: 3
  Median: 151.721s | MAD: 1.315s | Cutoff: 155.666s
  ✅ Saved filtered driver file: c:\Users\jacks\OneDrive\Desktop\progamming projects\toyota gr analysis\datasets_clean\cota2\driver_5.csv (10 valid laps)

Driver 7:
  Total laps: 12 | Valid: 8 | Removed: 4
  Median: 148.309s | MAD: 0.160s | Cutoff: 148.791s
  ✅ Saved filtered driver file: c:\Users\jacks\OneDrive\Desktop\progamming projects\toyota gr analysis\datasets_clean\cota2\driver_7.csv (8 valid laps)

Driver 8:
  Total laps: 13 | Valid: 10 | Removed: 3
  Median: 156.013s | MAD: 1.690s | Cutoff: 161.083s
  ✅ Saved filtered driver file: c:\Users\jacks\OneDrive\Desktop\progamming projects\toyota gr analysis\datasets_clean\cota2\dr

,DriverNumber,TotalLaps,ValidLaps,RemovedLaps,MedianLap(s),MAD(s),Cutoff(s)
0,2,9,7,2,149.7150,0.9220,152.4810
1,5,13,10,3,151.7210,1.3150,155.6660
2,7,12,8,4,148.3090,0.1605,148.7905
3,8,13,10,3,156.0130,1.6900,161.0830
4,11,13,10,3,153.6800,1.0200,156.7400
5,13,12,10,2,148.7835,0.4305,150.0750
6,14,12,9,3,151.9050,1.0755,155.1315
7,15,13,10,3,152.2120,1.4820,156.6580
8,16,13,10,3,149.6040,0.6520,151.5600
9,18,9,6,3,153.0610,0.2410,153.7840



✅ Saved filtered driver session stats to: c:\Users\jacks\OneDrive\Desktop\progamming projects\toyota gr analysis\datasets_clean\cota2\driver_session_stats.csv


,DriverNumber,Laps,BestLap(s),AvgLap(s),StdDev(s),S1Best,S2Best,S3Best,TheoreticalBest(s)
0,2,7,148.609,149.418,0.729,32.364,56.755,59.300,148.419
1,5,10,149.848,151.368,1.301,32.396,57.624,59.060,149.080
2,7,8,148.112,148.278,0.136,32.424,56.699,58.876,147.999
3,8,10,153.233,155.357,1.668,32.959,58.737,60.815,152.511
4,11,10,151.501,153.305,1.226,33.118,57.951,59.704,150.773
5,13,10,148.115,148.708,0.535,32.408,56.630,58.819,147.857
6,14,9,149.906,151.669,1.155,32.654,57.623,59.466,149.743
7,15,10,149.400,151.474,1.492,32.340,57.214,59.660,149.214
8,16,10,148.510,149.428,0.665,32.225,56.791,59.316,148.332
9,18,6,151.905,152.783,0.447,33.042,58.123,60.395,151.560


In [27]:
HARD_CUTOFF = 40
telemetry_file = os.path.join(OUTPUT_ROOT, "per_lap_telemetry_summary.csv")

# Load the telemetry summary
telemetry_df = pd.read_csv(telemetry_file)

# Extract driver number from vehicle_id (last number after dash)
telemetry_df['NUMBER'] = telemetry_df['vehicle_id'].str.split('-').str[-1].astype(int)

filtered_list = []

for driver, grp in telemetry_df.groupby('NUMBER'):
    median = grp['mean_throttle'].median()
    mad = (grp['mean_throttle'] - median).abs().median()
    # Define low-throttle cutoff (e.g., median - 3*MAD)
    cutoff = median - 3 * mad
    grp_filtered = grp[(grp['mean_throttle'] >= cutoff) & (grp['mean_throttle'] >= HARD_CUTOFF)]
    filtered_list.append(grp_filtered)
    print(f"Driver {driver}: median={median:.2f}, MAD={mad:.2f}, cutoff={cutoff:.2f}, removed={len(grp)-len(grp_filtered)} laps")

# Concatenate filtered groups
filtered_telemetry_df = pd.concat(filtered_list, ignore_index=True)

# Overwrite the original CSV
filtered_telemetry_df.to_csv(telemetry_file, index=False)

print(f"Filtered telemetry saved. Remaining rows: {len(filtered_telemetry_df)}")

Driver 2: median=73.32, MAD=4.65, cutoff=59.37, removed=6 laps
Driver 3: median=73.20, MAD=4.08, cutoff=60.96, removed=13 laps
Driver 5: median=71.46, MAD=4.15, cutoff=59.00, removed=7 laps
Driver 7: median=75.04, MAD=2.11, cutoff=68.72, removed=9 laps
Driver 8: median=62.85, MAD=4.96, cutoff=47.98, removed=7 laps
Driver 11: median=68.84, MAD=3.74, cutoff=57.61, removed=7 laps
Driver 13: median=76.23, MAD=3.61, cutoff=65.41, removed=9 laps
Driver 14: median=74.19, MAD=2.87, cutoff=65.59, removed=6 laps
Driver 15: median=68.51, MAD=5.42, cutoff=52.24, removed=7 laps
Driver 16: median=70.05, MAD=3.04, cutoff=60.94, removed=8 laps
Driver 18: median=69.36, MAD=2.81, cutoff=60.92, removed=5 laps
Driver 21: median=73.35, MAD=2.23, cutoff=66.66, removed=8 laps
Driver 31: median=75.40, MAD=3.53, cutoff=64.82, removed=7 laps
Driver 41: median=73.29, MAD=2.97, cutoff=64.39, removed=8 laps
Driver 46: median=77.36, MAD=4.07, cutoff=65.14, removed=7 laps
Driver 47: median=73.97, MAD=2.87, cutoff=65

In [28]:
# ---------------------- Setup ----------------------
telemetry_path = os.path.join(OUTPUT_ROOT, "per_lap_telemetry_summary.csv")
timestamp_path = os.path.join(OUTPUT_ROOT, "telemetry_per_timestamp.csv")

# Load telemetry summary
telemetry_df = pd.read_csv(telemetry_path)

# Extract driver NUMBER from vehicle_id (ignore leading zeros)
telemetry_df["NUMBER"] = telemetry_df["vehicle_id"].apply(
    lambda x: int(re.search(r"-(\d+)$", x).group(1))
)

# Create a lookup: NUMBER -> set of valid laps from telemetry
telemetry_laps = {
    num: set(grp["lap"].dropna().astype(int))
    for num, grp in telemetry_df.groupby("NUMBER")
}

# ---------------------- Clean + Sync driver CSVs ----------------------
valid_driver_laps = {}  # track which laps remain per driver

for number, valid_laps in telemetry_laps.items():
    driver_file = os.path.join(OUTPUT_ROOT, f"driver_{number}.csv")
    if not os.path.exists(driver_file):
        print(f"⚠️ Missing driver file for NUMBER {number}, skipping.")
        continue

    driver_df = pd.read_csv(driver_file)

    if "LAP_NUMBER" not in driver_df.columns:
        print(f"⚠️ File driver_{number}.csv has no LAP_NUMBER column, skipping.")
        continue

    before_count = len(driver_df)
    # Keep only laps present in telemetry
    driver_df = driver_df[driver_df["LAP_NUMBER"].isin(valid_laps)]

    # Drop unwanted columns
    cols_to_drop = [
        c for c in driver_df.columns
        if (
            c.strip().endswith("_IMPROVEMENT")
            or c.strip().endswith("_SECONDS")
            or c.strip().endswith("_elapsed")
            or c.strip().endswith("_LARGE")
        )
    ]
    driver_df = driver_df.drop(columns=cols_to_drop, errors="ignore")

    after_count = len(driver_df)

    # Save valid laps for timestamp filtering
    valid_driver_laps[number] = set(driver_df["LAP_NUMBER"].astype(int))

    # Overwrite cleaned file
    driver_df.to_csv(driver_file, index=False)

    print(f"✅ Updated driver_{number}.csv — kept {after_count}/{before_count} laps")

# ---------------------- Clean telemetry_per_timestamp.csv ----------------------
if os.path.exists(timestamp_path):
    print("\n🔍 Cleaning telemetry_per_timestamp.csv ...")
    ts_df = pd.read_csv(timestamp_path)

    # Extract driver number
    ts_df["NUMBER"] = ts_df["vehicle_id"].apply(
        lambda x: int(re.search(r"-(\d+)$", x).group(1))
    )

    before_ts = len(ts_df)

    # Keep rows where (NUMBER, lap) exists in valid_driver_laps
    mask = ts_df.apply(
        lambda row: row["NUMBER"] in valid_driver_laps
        and int(row["lap"]) in valid_driver_laps[row["NUMBER"]],
        axis=1
    )
    ts_df = ts_df[mask].copy()

    after_ts = len(ts_df)

    ts_df.to_csv(timestamp_path, index=False)
    print(f"✅ Cleaned telemetry_per_timestamp.csv — kept {after_ts}/{before_ts} rows")

else:
    print("⚠️ telemetry_per_timestamp.csv not found — skipping.")

print("\n✅ All driver files and telemetry timestamp data synchronized.")

✅ Updated driver_2.csv — kept 6/7 laps
⚠️ Missing driver file for NUMBER 3, skipping.
✅ Updated driver_5.csv — kept 8/10 laps
✅ Updated driver_7.csv — kept 6/8 laps
✅ Updated driver_8.csv — kept 8/10 laps
✅ Updated driver_11.csv — kept 8/10 laps
✅ Updated driver_13.csv — kept 8/10 laps
✅ Updated driver_14.csv — kept 7/9 laps
✅ Updated driver_15.csv — kept 8/10 laps
✅ Updated driver_16.csv — kept 8/10 laps
✅ Updated driver_18.csv — kept 5/6 laps
✅ Updated driver_21.csv — kept 10/10 laps
✅ Updated driver_31.csv — kept 8/10 laps
✅ Updated driver_41.csv — kept 8/10 laps
✅ Updated driver_46.csv — kept 6/8 laps
✅ Updated driver_47.csv — kept 10/10 laps
✅ Updated driver_51.csv — kept 7/9 laps
✅ Updated driver_55.csv — kept 8/10 laps
✅ Updated driver_57.csv — kept 6/8 laps
✅ Updated driver_71.csv — kept 8/10 laps
✅ Updated driver_72.csv — kept 8/10 laps
✅ Updated driver_73.csv — kept 8/10 laps
✅ Updated driver_80.csv — kept 4/8 laps
✅ Updated driver_86.csv — kept 8/10 laps
✅ Updated driver_88.

In [29]:
# Path setup
summary_path = os.path.join(OUTPUT_ROOT, "per_lap_telemetry_summary.csv")

# Load telemetry summary
summary_df = pd.read_csv(summary_path)

# Extract NUMBER from vehicle_id (ignore leading zeros)
summary_df["NUMBER"] = summary_df["vehicle_id"].apply(lambda x: int(re.search(r"-(\d+)$", x).group(1)))

filtered_frames = []

# Process each unique NUMBER
for number in summary_df["NUMBER"].unique():
    driver_file = os.path.join(OUTPUT_ROOT, f"driver_{number}.csv")
    if not os.path.exists(driver_file):
        print(f"⚠️ Missing driver file for NUMBER {number}, skipping.")
        continue

    driver_df = pd.read_csv(driver_file)
    available_laps = set(driver_df["LAP_NUMBER"].dropna().astype(int))

    # Keep only laps that exist in the driver's file
    filtered = summary_df[
        (summary_df["NUMBER"] == number) &
        (summary_df["lap"].isin(available_laps))
    ]

    filtered_frames.append(filtered)

# Combine and overwrite the original file
final_df = pd.concat(filtered_frames, ignore_index=True)
final_df.to_csv(summary_path, index=False)

print(f"✅ per_lap_telemetry_summary.csv successfully filtered and overwritten.")
print(f"Remaining rows: {len(final_df)}")

⚠️ Missing driver file for NUMBER 3, skipping.
✅ per_lap_telemetry_summary.csv successfully filtered and overwritten.
Remaining rows: 210
